In [54]:
#创建多层感知机 gluon
from mxnet import nd
from mxnet.gluon import nn

net=nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(256,activation="relu"))
    net.add(nn.Dense(10))
print(net)

Sequential(
  (0): Dense(None -> 256, Activation(relu))
  (1): Dense(None -> 10, linear)
)


In [70]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(10)

    def forward(self, x):
        return self.dense1(nd.relu(self.dense0(x)))

In [71]:

net2 = MLP()
print(net2)
net2.initialize()
x = nd.random.uniform(shape=(4,20))
y = net2(x)
y

MLP(
  (dense0): Dense(None -> 256, linear)
  (dense1): Dense(None -> 10, linear)
)



[[ 0.01599331  0.02209063  0.01248272 -0.02095865  0.02979716 -0.06946628
   0.04118977  0.01389062  0.01184154 -0.04527701]
 [-0.02399769  0.073647    0.00263377  0.00225113  0.00204536 -0.0674375
   0.04784648  0.02802196  0.04880201 -0.03907078]
 [-0.02892219  0.04902704 -0.00255     0.01337825 -0.00280327 -0.10357381
   0.04161734  0.03533161  0.08501115 -0.03772434]
 [ 0.01808984  0.04732734 -0.02078294  0.00740154 -0.00326047 -0.09708859
   0.03816021  0.00432553  0.00134935 -0.02988878]]
<NDArray 4x10 @cpu(0)>

In [75]:
# using class difine Sequential(顺序)
class Sequential(nn.Block):
    def __init__(self,**kwargs):
        super(Sequential,self).__init__(**kwargs)
    def add(self, block):
        self._children.append(block)#没有明白这个是什么意思
    def forward(self,x):
        for block in self._children:
            x = block(x)
        return x
        
        

In [77]:
net4 = Sequential()
with net4.name_scope():
    net4.add(nn.Dense(256, activation="relu"))
    net4.add(nn.Dense(10))

net4.initialize()
y = net4(x)
y


[[ 0.03776609  0.03380018  0.06707328 -0.09524883 -0.02695445  0.02556845
   0.05772297 -0.05677703  0.00282034  0.05922025]
 [ 0.06853435  0.08247618  0.08034877 -0.07636102 -0.03599111  0.00117045
   0.08831719 -0.0301978  -0.02525477  0.07853685]
 [ 0.07864901  0.08491036  0.0483099  -0.10731368 -0.00750124 -0.01912395
   0.10374747 -0.02697803 -0.03590386  0.11448428]
 [ 0.08836336  0.05076564  0.07970703 -0.08948221 -0.04017068  0.01738222
   0.07147183 -0.0474473  -0.00985061  0.07807525]]
<NDArray 4x10 @cpu(0)>

In [84]:
#class FancyMLP
class FancyMLP(nn.Block):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        with self.name_scope():
            self.dense=nn.Dense(256)
            self.weight = nd.random_uniform(shape=(256,20))
    
    def forward(self,x):#这个什么东西？
        x = nd.relu(self.dense(x))
        x = nd.relu(nd.dot(x, self.weight)+1)
        x = nd.relu(self.dense(x))
        return x
fancy_mlp = FancyMLP()
fancy_mlp.initialize()
y = fancy_mlp(x)
print(y.shape)

(4, 256)


In [86]:
#nn.Block和nn.Sequential的嵌套使用
class RecMLP(nn.Block):
    def __init__(self, **kwargs):
        super(RecMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        with self.name_scope():
            self.net.add(nn.Dense(256, activation="relu"))
            self.net.add(nn.Dense(128, activation="relu"))
            self.dense = nn.Dense(64)

    def forward(self, x):
        return nd.relu(self.dense(self.net(x)))

rec_mlp = nn.Sequential()
rec_mlp.add(RecMLP())
rec_mlp.add(nn.Dense(10))
print(rec_mlp)
        

Sequential(
  (0): RecMLP(
    (net): Sequential(
      (0): Dense(None -> 256, Activation(relu))
      (1): Dense(None -> 128, Activation(relu))
    )
    (dense): Dense(None -> 64, linear)
  )
  (1): Dense(None -> 10, linear)
)
